# Atmopsheric Retrievals with ExoCTK

*This notebook demonstrates the use of ExoCTK's `platon_wrapper` module.  As indicated by its name, this module is a wrapper around the `platon.retreiver.run_multinest` and `platon.retriever.run_emcee` methods, which runs nested sampling and EMCEE to retrieve atmospheric parameters, respectively.  For further information about `platon`, see the [project documentation](https://platon.readthedocs.io/en/latest/), the [API docs for the `retriever` module](https://platon.readthedocs.io/en/latest/source/platon.html#module-platon.retriever), or the [GitHub repository](https://github.com/ideasrule/platon).*

*Note that some of the examples provided below are minimal, bare-bones examples that are meant to show how users may use the software while not taking much computation time to complete.  The parameters used and the corresponding results are not indicitive of a true scientific use case.  For a more comprehensive and robust examples, see the `examples.py` module in the `exoctk.atmospheric_retrievals` subpackage.*

In [ ]:
# Required imports
from IPython.display import Image
import numpy as np
from platon.constants import R_sun, R_jup, M_jup
from exoctk.atmospheric_retrievals.platon_wrapper import PlatonWrapper

*The `PlatonWrapper` object requires the user to supply a dictionary containing initial guesses of parameters that they wish to fit.  Note that `Rs`, `Mp`, `Rp`, and `T` must be supplied, while the others are optional.*

*Note that `Rs` are in units of solar radii, `Mp` are in units of Jupiter masses, and `Rp` is is units of Jupiter radii.*

In [ ]:
params = {
    'Rs': 1.19,  # Required
    'Mp': 0.73,  # Required
    'Rp': 1.4,  # Required
    'T': 1200.0,  # Required
    'logZ': 0,  # Optional
    'CO_ratio': 0.53,  # Optional
    'log_cloudtop_P': 4,  # Optional
    'log_scatt_factor': 0,  # Optional
    'scatt_slope': 4,  # Optional
    'error_multiple': 1,  # Optional
    'T_star': 6091}  # Optional

*In order to perform the retrieval, users must instantiate a `PlatonWrapper` object and set the parameters*

In [ ]:
pw = PlatonWrapper()
pw.set_parameters(params)

*Users may add additional fitting constraints via the `fit_info` attribute.  Note that all of these optional.*

In [ ]:
R_guess = 1.4 * R_jup
T_guess = 1200
pw.fit_info.add_gaussian_fit_param('Rs', 0.02*R_sun)
pw.fit_info.add_gaussian_fit_param('Mp', 0.04*M_jup)
pw.fit_info.add_uniform_fit_param('Rp', 0.9*R_guess, 1.1*R_guess)
pw.fit_info.add_uniform_fit_param('T', 0.5*T_guess, 1.5*T_guess)
pw.fit_info.add_uniform_fit_param("log_scatt_factor", 0, 1)
pw.fit_info.add_uniform_fit_param("logZ", -1, 3)
pw.fit_info.add_uniform_fit_param("log_cloudtop_P", -0.99, 5)
pw.fit_info.add_uniform_fit_param("error_multiple", 0.5, 5)

*Prior to performing the retrieval, users must define `bins`, `depths`, and `errors` attributes.  The `bins` atribute must be a list of lists, with each element being the lower and upper bounds of the wavelenth bin.  The `depths` and `errors` attributes are both 1-dimensional `numpy` arrays.*

In [ ]:
pw.wavelengths = 1e-6*np.array([1.119, 1.1387])
pw.bins = [[w-0.0095e-6, w+0.0095e-6] for w in pw.wavelengths]
pw.depths = 1e-6 * np.array([14512.7, 14546.5])
pw.errors = 1e-6 * np.array([50.6, 35.5])

*With everything defined, users can now perform the retreival.  Users may choose to use the the EMCEE method (`emcee`) or the Multinested Sampling method (`multinest`)*

### EMCEE Method

In [ ]:
pw.retrieve('emcee')
pw.make_plot()

*Results are stored as a corner plot in the location given by the `output_plot` attribute*

In [ ]:
Image(filename='../atmospheric_retrievals/{}'.format(pw.output_plot))

### Mulinested Sampling Method

In [ ]:
pw.retrieve('multinest')
pw.save_results()
pw.make_plot()

*Results are stored as (1) a text file in the location given by the `output_results` attribute and (2) as a corner plot in the location given by the `output_plot` attribute*

In [ ]:
with open(pw.output_results, 'r') as f:
    results = f.readlines()
print(results)

In [ ]:
Image(filename='../atmospheric_retrievals/{}'.format(pw.output_plot))